In [1]:
import h5py
import tensorflow as tf
import numpy as np
from numpy import f2py

## Wrapping Fortran functions to Python

[Three ways to wrap](https://numpy.org/doc/stable/f2py/f2py.getting-started.html)

- Quick way (this requires Python 3.7 and NumPy 1.18): `python3 -m numpy.f2py -c eos-t.f90 -m eos_t`


In [3]:
from platform import python_version
print(python_version())

3.7.10


In [4]:
import eos_t

In [5]:
eos_t.atomic_rates.tabulate_rates()
print(eos_t.atomic_rates.alphahp)

[1.26271073e-10 1.25558019e-10 1.24848791e-10 ... 6.57445059e-19
 6.47458444e-19 6.37622769e-19]


## Testing speed of tensor access

It takes too long to access a tensor's entries one-by-one; I need to avoid this when I'm passing fields into the EOS or optical depth routines

In [12]:
def time_stats(duration):
    rate = duration / n**2
    print("Duration:", np.round(duration, 4))
    print("Hours needed to access 1024^3 entries:", 
      np.round(rate * 1024**3 / 3600, 4))

In [17]:
import time

n = 10
field = tf.zeros((n,n))
total = 0

start = time.time()

for i in range(n):
    for j in range(n):
        total += field[i,j]

duration = time.time() - start
time_stats(duration)

Duration: 0.0252
Hours needed to access 1024^3 entries: 75.242


### Nested loops

In [24]:
count = 15
n = 10

start = time.time()

# outer loop
i = tf.constant(0)
condition1 = lambda i, count: tf.less(i, n)

def body1(i, r): # index i and result r
    # inner loop
    j = tf.constant(0)
    condition2 = lambda i,j,r: tf.less(j, n)
    
    def body2(i, j, r):
        r += field[i,j]
        return i, tf.add(j, 1), r
    
    i, j, r = tf.while_loop(condition2, body2, loop_vars=[i,j, r])
    
    # increment r
    return tf.add(i, 1), r

# do the loop:
r = tf.while_loop(condition1, body1, [i, count])
print(r)

duration = time.time() - start
time_stats(duration)

(<tf.Tensor: shape=(), dtype=int32, numpy=10>, <tf.Tensor: shape=(), dtype=float32, numpy=15.0>)
Duration: 0.0686
Hours needed to access 1024^3 entries: 204.7353


## Testing tf gradients

In [13]:
tf.math.pow(z, 2)

<tf.Tensor: shape=(), dtype=float32, numpy=9.0>

In [11]:
z = tf.constant(3.)
with tf.GradientTape() as tape:
    tape.watch(z) # without this line, grad is None
    a = tf.divide(1, z+1)
    print('a:', a)
    
grad = tape.gradient(a, z)
print('grad:', grad)

a: tf.Tensor(0.25, shape=(), dtype=float32)
grad: tf.Tensor(-0.0625, shape=(), dtype=float32)


# Load in the data

In [5]:
filename = "../../../../../cscratch1/sd/jupiter/sim2_z3_FGPA_cgs.h5"

#snap = h5py.File(filename,'r')

## Testing classes

In [6]:
import snapshot

snap = snapshot.Snapshot(filename)

In [7]:
# test snapshot methods
temp = snap.read_field('/native_fields/temperature')
temp[42,42,42]

<tf.Tensor: shape=(), dtype=float32, numpy=5971.877>

In [8]:
rhob = snap.read_field('/native_fields/baryon_density')
rhob[42,42,42]

<tf.Tensor: shape=(), dtype=float32, numpy=0.22918068>

In [4]:
# test the universe class (from universe.py)
u = snap.universe
chi = 10
z = 1
u.chi_to_proper_cgs(chi, z)

<tf.Tensor: shape=(), dtype=float64, numpy=2.2856888888888885e+25>

## Metadata

In [6]:
shape = snap['domain'].attrs['shape']
size = snap['domain'].attrs['size']

z = snap['universe'].attrs['redshift']
omega_b = snap['universe'].attrs['omega_b']
omega_m = snap['universe'].attrs['omega_m']
omega_l = snap['universe'].attrs['omega_l']
h = snap['universe'].attrs['hubble']

scale_factor = 1.0 / (1.0 + z)

In [ ]:
# TODO: (possibly) implement a grid class
# grid = new Grid(shape[0], shape[1], shape[2], size[0], size[1], size[2])

In [7]:
#snap.close()